In [1]:
from Analysis.systems import kitaev_chain,kramers_chain,kitaev_chain_spinful

import FockSystem.FockSystem as fst
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import time

In [2]:
from IPython.core.display import Markdown
from IPython.display import display

In [3]:
import Analysis.systems as kt

# Example

In [4]:
from FockSystem.FockSystemSparse import FockStatesSparse

In [5]:
N = 9
MU,U, ECT,CAR = kitaev_chain_spinful(N)
H = MU + CAR + ECT + U
H[MU] = 10
H[CAR] = 25
H[ECT] = 13
H[U] = 8

In [ ]:
normal_basis = fst.FockStates(N)
sparse_basis = fst.FockStatesSparse(N)

In [ ]:
input_vector = np.array([np.random.randint(2) for _ in range(4**N)], dtype=np.float64)
comp = np.array(input_vector,dtype=complex)

In [ ]:
sparse_matr = H[normal_basis].to_sparse_csr()
%time sparse_matr @ input_vector

In [ ]:
test_sparse = H[sparse_basis] @ input_vector
test_normal = H[normal_basis] @ comp
np.all(test_sparse==test_normal)

In [ ]:
build_normal_times = []
matvec_normal_times = []
memory_normal = []

build_sparse_times = []
matvec_sparse_times = []
memory_sparse = []

### Test Sparse build
N_range_sparse = np.arange(2,20)
print("Testing sparse build")

for n in N_range_sparse:
    print(n)
    ### Make H
    MU,U, ECT,CAR = kitaev_chain_spinful(n)
    H = MU + CAR + ECT + U
    H[MU] = 10
    H[CAR] = 25
    H[ECT] = 13
    H[U] = 8

    ### Test Normal and Sparse
    summed_constr = 0
    repeated_constr = 5

    for _ in range(repeated_constr):
        ### CONSTRUCTION
        sparse_basis = fst.FockStatesSparse(int(n))
        start = time.perf_counter()
        sparse_construct = H[sparse_basis]
        end = time.perf_counter()
        summed_constr += (end-start)        
        del sparse_basis
    memory_sparse.append(sparse_construct.rc_indices.nbytes)
    del sparse_construct
    build_sparse_times.append(summed_constr/repeated_constr)

### Test Sparse MATRIX - VECTOR
N_range_sparse_mv = np.arange(2,12)
print("Testing sparse matvec")

for n in N_range_sparse_mv:
    print(n)

    ### Make H
    MU,U, ECT,CAR = kitaev_chain_spinful(n)
    H = MU + CAR + ECT + U
    H[MU] = 10
    H[CAR] = 25
    H[ECT] = 13
    H[U] = 8
    sparse_basis = fst.FockStatesSparse(int(n))
    sparse_construct = H[sparse_basis]

    ### Test Normal and Sparse
    summed_constr = 0
    repeated_constr = 1
    for _ in range(repeated_constr):
        test_vector = np.array([np.random.randint(2) for _ in range(4**n)], dtype=np.float64)
        ### CONSTRUCTION
        start = time.perf_counter()
        res = sparse_construct @ test_vector
        end = time.perf_counter()
        summed_constr += (end-start)        
        del test_vector
        del res
    del sparse_basis
    del sparse_construct
    matvec_sparse_times.append(summed_constr/repeated_constr)

### Test Normal build
N_range_normal = np.arange(2,11)
print("Testing normal build")
for n in N_range_normal:
    print(n)
    ### Make H
    MU,U, ECT,CAR = kitaev_chain_spinful(n)
    H = MU + CAR + ECT + U
    H[MU] = 10
    H[CAR] = 25
    H[ECT] = 13
    H[U] = 8

    ### Test Normal
    summed = 0
    if n < 5:
        repeated = 1
    else:
        repeated = 1
    for _ in range(repeated):
        normal_basis = fst.FockStates(int(n))
        start = time.perf_counter()
        normal_construct = H[normal_basis]
        end = time.perf_counter()
        summed += (end-start)
        del normal_basis
        del normal_construct
    build_normal_times.append(summed/repeated)


### Test Normal  MATRIX - VECTOR
N_range_normal_mv = np.arange(2,11)
print("Testing normal matvec")
for n in N_range_normal_mv:
    print(n)
    ### Make H
    MU,U, ECT,CAR = kitaev_chain_spinful(n)
    H = MU + CAR + ECT + U
    H[MU] = 10
    H[CAR] = 25
    H[ECT] = 13
    H[U] = 8
    normal_basis = fst.FockStates(int(n))
    normal_construct = H[normal_basis]

    ### Test Normal
    summed = 0
    if n < 8:
        repeated = 1
    else:
        repeated = 1
    for _ in range(repeated):
        csr_matr = normal_construct.to_sparse_csr()
        memory_csr = csr_matr.data.nbytes + csr_matr.indices.nbytes + csr_matr.indptr.nbytes
        memory_normal.append(memory_csr)
        test_vector = np.array([np.random.randint(2) for _ in range(4**n)],  dtype=np.float64)
        
        start = time.perf_counter()
        res = csr_matr @ test_vector
        end = time.perf_counter()
        
        summed += (end-start)
        del csr_matr
        del test_vector
        del res
    
        
    del normal_basis
    del normal_construct
    matvec_normal_times.append(summed/repeated)


Testing sparse build
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Testing sparse matvec
2
3
4
5
6
7
8
9
10
11
Testing normal build
2
3
4
5
6
7
8
9
10
Testing normal matvec
2
3
4
5
6
7
8
9
10


In [ ]:
fig,axs = plt.subplots(ncols = 3, figsize=(10,3))

### Build Time
axs[0].set_yscale('log')
axs[0].plot(N_range_normal*2,build_normal_times, label = 'sparse', marker='x')
axs[0].plot(N_range_sparse*2, build_sparse_times, label = 'very sparse', marker='o')

axs[0].set_xlabel('# Fermions')
axs[0].set_ylabel('Time (s)')
axs[0].legend(loc='lower right',ncols=2,fontsize=8)
axs[0].set_title('Build time')

### Matrix - vector product
axs[1].set_yscale('log')
axs[1].set_ylabel('Time (s)')
axs[1].set_xlabel('# Fermions')

axs[1].plot(N_range_normal_mv*2, matvec_normal_times, label = 'scp.csr_matrix', marker='x')
axs[1].plot(N_range_sparse_mv*2, matvec_sparse_times, label = 'very sparse', marker='o')
axs[1].set_title('Matrix-vector product')
axs[1].legend(loc='upper left',ncols=1,fontsize=8)

### Memory requirement (excl. vector)
axs[2].set_yscale('log')
axs[2].set_title('Memory (excl. vector)')

axs[2].plot(N_range_normal_mv*2, memory_normal, label = 'scp.csr_matrix', marker='x')
axs[2].plot(N_range_sparse*2, memory_sparse, label = 'very sparse', marker='o')
axs[2].legend(loc='upper left',ncols=1,fontsize=8)
axs[2].set_ylabel('Bytes')
axs[2].set_xlabel('# Fermions')
plt.tight_layout()

## Visualise

In [ ]:
def print_md(item):
    display(Markdown(item._repr_markdown_()))

In [ ]:
## Build the Hamiltonian
fig,axs = plt.subplots(ncols=4,figsize = (14,5))

for idx,N in enumerate(range(2,6)):
    ## Build the Hamiltonian
    MU,CAR,ECT = kitaev_chain(N)
    H = MU + CAR +ECT
    H[MU] = 1
    H[CAR] = 5
    H[ECT] = 2

    
    axs[idx].matshow(np.real(H[basis].to_array()))

plt.tight_layout()

In [ ]:
def generate_minimal_data_tupels(H_base,N):
    basis = fst.FockStates(2)
    big_N = 4**N
    start_tuples, all_tuples = [],[]

    h_vals = []
    h_data = []
    for r,c,v,p,t in zip(H_base[basis].rows,H_base[basis].cols,H_base[basis].values, H_base[basis].parities, H_base[basis].type_strings):
        if r!=c:
            start_tuples.append((r,c,v*p,t))

    for solo_tuple in start_tuples:
        start_i,start_j = solo_tuple[0],solo_tuple[1]
        base_type = solo_tuple[3]

        for idx in range(2,N+1):
            val = solo_tuple[2]
            if (start_i < big_N and start_j < big_N):
                h_vals.append(val)
                h_data.append([start_i,start_j, 4**(idx-2), int(big_N/(4**idx)), 4**idx])
            if start_i != start_j:
                start_i <<= 2
                start_j <<= 2
            else:
                break
           
    return np.array(h_vals, dtype=np.float64),np.array(h_data,dtype=np.int64)

## Systematic approach

In [ ]:
N = 5
MU,CAR,ECT = kitaev_chain(N)
H = MU + CAR + ECT
H[MU] = 5
H[CAR] = 25
H[ECT] = 13

In [ ]:
c_dwn = fst.OperSequence(1)
c_up = fst.OperSequence(3)

In [ ]:
def construct_sparse_tuples(H, N):
    big_N = 4**N
    ## (1) Map each subsequence to a base element
    base_set_list = [] ## Set of minimal elements
    order_list = [] ## Order of the operator
    for i in range(len(H.oper_list)):
        subseq = H[i]
        sub_list = np.array(subseq.oper_list[0])
        sub_list -= ((np.min(sub_list) >> 2) * 4) ## Shift sequence to minimal form
        oper_order = (np.max(sub_list)>>2)-(np.min(sub_list)>>2) ## calculate 'order' of the sequence
        
        if list(sub_list) not in base_set_list:
            base_set_list.append(list(sub_list))
            order_list.append(int(oper_order))
            
    ## (2) Use the base set and orders to construct the minimal info needed for matvec
    h_vals = []
    h_types = []
    h_rc_data = []

    for order,base_set in zip(order_list,base_set_list):
        oper_as_sequence = fst.OperSequence(list(base_set[::-1]))        
        ## Calculate the relevant indices in the smallest possible subsystem
        basis = fst.FockStates(order+1)
        operseq_data = oper_as_sequence[basis]
        start_tuples=[]

        ## Shift the subsystem over the entire system
        ## Checking if the term indeed exists along the way and grabbing the correct values
        for row,col,parity in zip(operseq_data.rows,operseq_data.cols,operseq_data.parities):
            start_i = row
            start_j = col
            ## Can shift a sequence N-order times before going out of bounds
            for shift_idx in range(0, N-order):
                shifted_sequence = oper_as_sequence >> shift_idx
                ## Check if the sequence actually should be stored (store only if original H contains it)
                if shifted_sequence in H:
                    val = H.weights[H.oper_list.index(shifted_sequence.oper_list[0])]*parity ## grab value of the operator
                    type_string = shifted_sequence.oper_list_to_str(shifted_sequence.oper_list[0])
                    if (start_i < big_N and start_j < big_N):
                        if start_i == start_j:
                            h_vals.append(val/2)
                        else:
                            h_vals.append(val)

                        ## Stores: [starting_i, starting_j, number of small reps, number of big reps, spacing between big reps]
                        ## The latter 3 depend on the order of the sequence, the size of the array and the shifts that have been done
                        h_rc_data.append([start_i,start_j, 4**(shift_idx), int(4**(N-(order+1)-shift_idx)), 4**(shift_idx+order+1)]) 
                        h_types.append(type_string)
                start_i <<= 2
                start_j <<= 2
    return np.array(h_vals, dtype=np.float64),np.array(h_rc_data,dtype=np.int64), np.array(h_types)

        

In [ ]:
## Build the Hamiltonian
MU_2,CAR_2,ECT_2 = kt.kitaev_chain(2)
H_base = MU_2 + CAR_2 +ECT_2
H_base[MU_2] = 0
H_base[CAR_2] = 25
H_base[ECT_2] = 13

In [ ]:
N = 4
h_vals,h_data,h_types = construct_sparse_tuples(H,N)

In [ ]:
N = 7
MU,ECT,CAR = kramers_chain(N)
H = MU + CAR + ECT
H[MU] = 1
H[CAR] = 25
H[ECT] = 13

test_val,test_data,h_types = construct_sparse_tuples(H, N)
#basis=fst.FockStates(N)


In [ ]:
input_vector = np.array([np.random.randint(2) for _ in range(4**N)], dtype=np.float64)

In [ ]:
%time sparse_matrix = H[basis].to_sparse_coo()
%time matr = csr_matrix(sparse_matrix)

In [ ]:
# Compute memory usage
memory_bytes = (
    matr.data.nbytes +        # non-zero values
    matr.indices.nbytes +     # row indices
    matr.indptr.nbytes        # column pointers
)


print(f"Memory footprint: {memory_bytes / 1024**2:.2f} MB")


In [ ]:
%time matvec_fast(input_vector, test_data,test_val)
%time H[basis] @ np.array(input_vector, dtype=complex)
%time matr @ input_vector

In [ ]:
test = matvec_fast(input_vector, test_data,test_val)
test2 = H[basis] @ np.array(input_vector, dtype=complex)
np.all(test==test2)

In [ ]:
%load_ext cython

In [ ]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp

from cython.parallel import prange

import numpy as np
cimport numpy as cnp
from cython cimport boundscheck, wraparound
from libc.stdlib cimport malloc, free


@boundscheck(False)
@wraparound(False)
cdef int fill_memory(double[:] input_vec, double[:] output_vec,
                long start_i, long start_j, long small_rep, long big_rep, long big_rep_spacing, double val) nogil:
    cdef int i, j
    cdef long i_val, j_val
    cdef double tmp
   
    for i in range(big_rep):
        for j in range(small_rep):
            i_val = start_i + i * big_rep_spacing + j
            j_val = start_j + i * big_rep_spacing + j
            output_vec[i_val] += input_vec[j_val] * val
            output_vec[j_val] += input_vec[i_val] * val
    return 0

# Parallel loop function in Cython using prange (OpenMP)
@boundscheck(False)
@wraparound(False)
cpdef matvec_fast(cnp.ndarray[double, ndim=1] input_vec, long long[:,:] H_base_data, cnp.ndarray[double, ndim=1] H_base_vals):
    cdef int counter=0
    cdef int n_tuples = H_base_data.shape[0]
    cdef cnp.ndarray[double,ndim=1] output_vec = np.zeros_like(input_vec,order='C')
    cdef long base_i, base_j, small_rep, big_rep, big_rep_spacing
    cdef double base_value = 0
    cdef int n_threads = 0

    for counter in range(n_tuples):
        base_i = H_base_data[counter][0]
        base_j = H_base_data[counter][1]
        small_rep = H_base_data[counter][2]
        big_rep = H_base_data[counter][3]
        big_rep_spacing = H_base_data[counter][4]
        base_value = H_base_vals[counter]
        if base_value != 0:
            fill_memory(input_vec,output_vec, base_i,base_j, small_rep,big_rep, big_rep_spacing,base_value)

    return output_vec


In [ ]:
## Build the Hamiltonian
MU_2,CAR_2,ECT_2 = kt.kitaev_chain(2)
H_base = MU_2 + CAR_2 +ECT_2
H_base[MU_2] = 0
H_base[CAR_2] = 25
H_base[ECT_2] = 13

In [ ]:
N = 7
h_vals,h_data = generate_minimal_data_tupels(H_base,N)

In [ ]:
N = 7
h_vals,h_data = generate_minimal_data_tupels(H_base,N)

MU,CAR,ECT = kitaev_chain(N)
H = MU + CAR + ECT
H[MU] = 0
H[CAR] = 25
H[ECT] = 13
basis=fst.FockStates(N)
H[basis]

In [ ]:
input_vector = np.array([np.random.randint(2) for _ in range(4**N)], dtype=np.float64)
#compl_vector = np.array(input_vector, dtype=complex)

In [ ]:
%time test = matvec_fast(input_vector, h_data,h_vals)


In [ ]:
test = matvec_fast(input_vector, h_data,h_vals)
test2 = H[basis] @ np.array(input_vector, dtype=complex)
np.all(test==test2)

In [ ]:
from functools import partial
from scipy.sparse.linalg import LinearOperator, eigsh
from scipy.sparse import csc_matrix
sparse_func = partial(matvec_fast, H_base_data = h_data, H_base_vals = h_vals)

In [ ]:
%time sparse_matrix = H[basis].to_sparse_coo()
%time matr = csr_matrix(sparse_matrix)

In [ ]:
## Standard numpy matrix vector product
#print("Standard full matrix-vector product")
#%timeit H[basis].to_array() @ compl_vector

## Sparse matrix-vector product
print("Sparse matrix-vector product")
%time H[basis] @ compl_vector

print("CSR_matrix")
%time matr @ compl_vector

## Memory-less matrix-vector product
print("Memory-less matrix-vector product")
%time matvec_fast(input_vector, h_data,h_vals)



In [ ]:
%timeit eigsh(matr, k=10, which='SA')

In [ ]:
### With Sparse Coo
sparse_matrix = H[basis].to_sparse_coo()
E_odd,phi_odd = eigsh(sparse_matrix,k=10,which='SA')
print(E_odd)

%timeit eigsh(sparse_matrix, k=5, which='SA')

In [ ]:
### With Sparse cython loop
M_even = LinearOperator((len(input_vector), len(input_vector)), 
                               matvec= H[basis].get_sparse_func(), dtype=complex)

E_odd, phi_odd = eigsh(M_even, k=5, which='SA')
print(E_odd)
%timeit eigsh(M_even, k=5, which='SA')


In [ ]:
### With Matvec Fast
M_even = LinearOperator((len(input_vector), len(input_vector)), matvec= sparse_func, dtype=np.float64)

E_odd, phi_odd = eigsh(M_even, k=5, which='SA')
print(E_odd)
%timeit eigsh(M_even, k=5, which='SA')


In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
t = csr_matrix((data, (row, col)), shape=(3, 3)).toarray()

In [ ]:
t

## Spin

In [ ]:
## Build the Hamiltonian
MU_2,U_2, CAR_2,ECT_2 = kt.kitaev_chain_spinful(2)
H_base = MU_2 +U_2 +  CAR_2 +ECT_2
H_base[MU_2] = 0
H_base[U_2] = 0
H_base[CAR_2] = 15
H_base[ECT_2] = 20

In [ ]:
start = time()
N = 10
h_vals,h_data = generate_minimal_data_tupels(H_base,N)

MU,U,CAR,ECT = kt.kitaev_chain_spinful(N)
H = MU +U+ CAR + ECT
H[MU] = 5
H[U] = 5
H[CAR] = 15
H[ECT] = 20
basis=fst.FockStates(N)
H[basis]
end = time()
print(f'Generated system in {np.round(end-start)}s')

In [ ]:
input_vector=  np.array([np.random.randint(2) for _ in range(4**N)], dtype=np.float64)
compl_vector = np.array(input_vector, dtype=complex)

In [ ]:
test = matvec_fast(input_vector, h_data,h_vals)
test2 = H[basis] @ np.array(input_vector, dtype=complex)
np.all(test==test2)

In [ ]:
sparse_matrix = H[basis].to_sparse_coo()
matr = csc_matrix(sparse_matrix)

In [ ]:
# Compute memory usage
memory_bytes = (
    matr.data.nbytes +        # non-zero values
    matr.indices.nbytes +     # row indices
    matr.indptr.nbytes        # column pointers
)

print(f"Memory footprint: {memory_bytes / 1024**2:.2f} MB")


In [ ]:
# Sparse matrix-vector product
print("Sparse matrix-vector product")
%time H[basis] @ compl_vector

print("CSC_matrix")
%time matr @ compl_vector

## Memory-less matrix-vector product
print("Memory-less matrix-vector product")
%time matvec_fast(input_vector, h_data,h_vals)

In [ ]:
start = time()

sparse_func = partial(matvec_fast, H_base_data = h_data, H_base_vals = h_vals)
M_even = LinearOperator((len(input_vector), len(input_vector)), 
                               matvec= sparse_func, dtype=np.float64)

#E_odd, phi_odd = eigsh(M_even, k=10, which='SA')
E,phi= eigsh(M_even, k=5, which='SA')
print(E)
end = time()
print(f'Solved in {np.round(end-start,2)} seconds')

In [ ]:
del phi

In [ ]:
start = time()

sparse_func = partial(matvec_fast, H_base_data = h_data, H_base_vals = h_vals)
M_even = LinearOperator((len(input_vector), len(input_vector)), 
                               matvec= sparse_func, dtype=np.float64)

#E_odd, phi_odd = eigsh(M_even, k=10, which='SA')
E = eigsh(M_even, k=5, which='SA',return_eigenvectors=False)
print(E)
end = time()
print(f'Solved in {np.round(end-start,2)} seconds')

In [ ]:
start = time()
M_even = LinearOperator((len(input_vector), len(input_vector)), 
                               matvec= H[basis].get_sparse_func(), dtype=complex)

#E_odd, phi_odd = eigsh(M_even, k=10, which='SA')
E,phi = eigsh(M_even, k=5, which='SA')
end = time()
print(f'Solved in {np.round(end-start,2)} seconds')
print(E)